[View in Colaboratory](https://colab.research.google.com/github/wonwooddo/Pytorch_tutorial/blob/master/04.utils_tensorboard.ipynb)

In [1]:
!pip install torch
!pip install torchvision
!pip install Pillow==4.0.0
!pip install PIL
!pip install image


torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Using cached https://files.pythonhosted.org/packages/00/49/a0483e7308b4b04b5a898789911dbb876d9fea54e7df0453915e47744cfd/Pillow-5.1.0-cp27-cp27mu-manylinux1_x86_64.whl
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
  Using cached https://files.pythonhosted.org/packages/89/99/0e3522a9764fe371bf9f7729404b1ef7d9c4fc49cbe5f1761c6e07812345/Pillow-4.0.0-cp27-cp27mu-manylinux1_x86_64.whl
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.1.0
    Uninstalling Pillow-5.1.0:
      Successfully uninstalled Pillow-5.1.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.


In [0]:
import PIL.Image

In [0]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import tensorflow as tf
import numpy as np

import scipy.misc 
try:
    from StringIO import StringIO  # Python 2.7
except ImportError:
    from io import BytesIO         # Python 3.x


In [0]:
class Logger(object):
    
    def __init__(self, log_dir):
        """Create a summary writer logging to log_dir."""
        self.writer = tf.summary.FileWriter(log_dir)

    def scalar_summary(self, tag, value, step):
        """Log a scalar variable."""
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=value)])
        self.writer.add_summary(summary, step)

    def image_summary(self, tag, images, step):
        """Log a list of images."""

        img_summaries = []
        for i, img in enumerate(images):
            # Write the image to a string
            try:
                s = StringIO()
            except:
                s = BytesIO()
            scipy.misc.toimage(img).save(s, format="png")

            # Create an Image object
            img_sum = tf.Summary.Image(encoded_image_string=s.getvalue(),
                                       height=img.shape[0],
                                       width=img.shape[1])
            # Create a Summary value
            img_summaries.append(tf.Summary.Value(tag='%s/%d' % (tag, i), image=img_sum))

        # Create and write Summary
        summary = tf.Summary(value=img_summaries)
        self.writer.add_summary(summary, step)
        
    def histo_summary(self, tag, values, step, bins=1000):
        """Log a histogram of the tensor of values."""

        # Create a histogram using numpy
        counts, bin_edges = np.histogram(values, bins=bins)

        # Fill the fields of the histogram proto
        hist = tf.HistogramProto()
        hist.min = float(np.min(values))
        hist.max = float(np.max(values))
        hist.num = int(np.prod(values.shape))
        hist.sum = float(np.sum(values))
        hist.sum_squares = float(np.sum(values**2))

        # Drop the start of the first bin
        bin_edges = bin_edges[1:]

        # Add bin edges and counts
        for edge in bin_edges:
            hist.bucket_limit.append(edge)
        for c in counts:
            hist.bucket.append(c)

        # Create and write Summary
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, histo=hist)])
        self.writer.add_summary(summary, step)
        self.writer.flush()

In [0]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# MNIST dataset 
dataset = torchvision.datasets.MNIST(root='../../data', 
                                     train=True, 
                                     transform=transforms.ToTensor(),  
                                     download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=dataset, 
                                          batch_size=100, 
                                          shuffle=True)

In [0]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size=784, hidden_size=500, num_classes=10):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet().to(device)


In [0]:
logger = Logger('./logs')

In [0]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)  

In [0]:
def register_extension(id, extension):
    PIL.Image.EXTENSION[extension.lower()] = id.upper()
PIL.Image.register_extension = register_extension
def register_extensions(id, extensions):
    for extension in extensions:
        register_extension(id, extension)
PIL.Image.register_extensions = register_extensions

In [0]:
data_iter = iter(data_loader)
iter_per_epoch = len(data_loader)
total_step = 50000

In [0]:
# Start training
for step in range(total_step):
    
    # Reset the data_iter
    if (step+1) % iter_per_epoch == 0:
        data_iter = iter(data_loader)

    # Fetch images and labels
    images, labels = next(data_iter)
    images, labels = images.view(images.size(0), -1).to(device), labels.to(device)
    
    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Compute accuracy
    _, argmax = torch.max(outputs, 1)
    accuracy = (labels == argmax.squeeze()).float().mean()

    if (step+1) % 100 == 0:
        print ('Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(step+1, total_step, loss.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

        # 1. Log scalar values (scalar summary)
        info = { 'loss': loss.item(), 'accuracy': accuracy.item() }

        for tag, value in info.items():
            logger.scalar_summary(tag, value, step+1)

        # 2. Log values and gradients of the parameters (histogram summary)
        for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            logger.histo_summary(tag, value.data.cpu().numpy(), step+1)
            logger.histo_summary(tag+'/grad', value.grad.data.cpu().numpy(), step+1)

        # 3. Log training images (image summary)
        info = { 'images': images.view(-1, 28, 28)[:10].cpu().numpy() }

        for tag, images in info.items():
            logger.image_summary(tag, images, step+1)

Step [100/50000], Loss: 2.1866, Acc: 0.45
Step [200/50000], Loss: 2.0479, Acc: 0.74
Step [300/50000], Loss: 1.9544, Acc: 0.73
Step [400/50000], Loss: 1.8005, Acc: 0.77
Step [500/50000], Loss: 1.6497, Acc: 0.82
Step [600/50000], Loss: 1.5552, Acc: 0.79
Step [700/50000], Loss: 1.4571, Acc: 0.80
Step [800/50000], Loss: 1.3430, Acc: 0.74
Step [900/50000], Loss: 1.1478, Acc: 0.84
Step [1000/50000], Loss: 1.1634, Acc: 0.82
Step [1100/50000], Loss: 1.0487, Acc: 0.85
Step [1200/50000], Loss: 0.9536, Acc: 0.86
Step [1300/50000], Loss: 0.9910, Acc: 0.80
Step [1400/50000], Loss: 0.8658, Acc: 0.84
Step [1500/50000], Loss: 0.9392, Acc: 0.82
Step [1600/50000], Loss: 0.8197, Acc: 0.87
Step [1700/50000], Loss: 0.8122, Acc: 0.80
Step [1800/50000], Loss: 0.7955, Acc: 0.82
Step [1900/50000], Loss: 0.6013, Acc: 0.90
Step [2000/50000], Loss: 0.7255, Acc: 0.88
Step [2100/50000], Loss: 0.6816, Acc: 0.85
Step [2200/50000], Loss: 0.7022, Acc: 0.81
Step [2300/50000], Loss: 0.6438, Acc: 0.85
Step [2400/50000], L

Step [6800/50000], Loss: 0.3995, Acc: 0.86
Step [6900/50000], Loss: 0.2302, Acc: 0.94
Step [7000/50000], Loss: 0.4002, Acc: 0.92
Step [7100/50000], Loss: 0.2658, Acc: 0.93
Step [7200/50000], Loss: 0.2728, Acc: 0.91
Step [7300/50000], Loss: 0.4588, Acc: 0.88
Step [7400/50000], Loss: 0.3009, Acc: 0.92
Step [7500/50000], Loss: 0.4464, Acc: 0.86
Step [7600/50000], Loss: 0.4066, Acc: 0.89
Step [7700/50000], Loss: 0.4005, Acc: 0.93
Step [7800/50000], Loss: 0.4157, Acc: 0.89
Step [7900/50000], Loss: 0.3094, Acc: 0.91
Step [8000/50000], Loss: 0.3735, Acc: 0.90
Step [8100/50000], Loss: 0.5349, Acc: 0.83
Step [8200/50000], Loss: 0.3989, Acc: 0.92
Step [8300/50000], Loss: 0.2864, Acc: 0.96
Step [8400/50000], Loss: 0.2345, Acc: 0.93
Step [8500/50000], Loss: 0.3184, Acc: 0.91
Step [8600/50000], Loss: 0.3708, Acc: 0.90
Step [8700/50000], Loss: 0.3492, Acc: 0.94
Step [8800/50000], Loss: 0.3046, Acc: 0.94
Step [8900/50000], Loss: 0.3306, Acc: 0.94
Step [9000/50000], Loss: 0.3273, Acc: 0.92
Step [9100/

Step [13500/50000], Loss: 0.2230, Acc: 0.91
Step [13600/50000], Loss: 0.3812, Acc: 0.90
Step [13700/50000], Loss: 0.3054, Acc: 0.93
Step [13800/50000], Loss: 0.2774, Acc: 0.92
Step [13900/50000], Loss: 0.2062, Acc: 0.93
Step [14000/50000], Loss: 0.2969, Acc: 0.93
Step [14100/50000], Loss: 0.2161, Acc: 0.95
Step [14200/50000], Loss: 0.2564, Acc: 0.91
Step [14300/50000], Loss: 0.1721, Acc: 0.96
Step [14400/50000], Loss: 0.2300, Acc: 0.91
Step [14500/50000], Loss: 0.3334, Acc: 0.88
Step [14600/50000], Loss: 0.3109, Acc: 0.90
Step [14700/50000], Loss: 0.2670, Acc: 0.92
Step [14800/50000], Loss: 0.2356, Acc: 0.93
Step [14900/50000], Loss: 0.3205, Acc: 0.93
Step [15000/50000], Loss: 0.1490, Acc: 0.96
Step [15100/50000], Loss: 0.2802, Acc: 0.91
Step [15200/50000], Loss: 0.2737, Acc: 0.93
Step [15300/50000], Loss: 0.1788, Acc: 0.95
Step [15400/50000], Loss: 0.1793, Acc: 0.96
Step [15500/50000], Loss: 0.2460, Acc: 0.92
Step [15600/50000], Loss: 0.1833, Acc: 0.96
Step [15700/50000], Loss: 0.1473

Step [20200/50000], Loss: 0.1201, Acc: 0.97
Step [20300/50000], Loss: 0.3037, Acc: 0.92
Step [20400/50000], Loss: 0.2542, Acc: 0.95
Step [20500/50000], Loss: 0.2016, Acc: 0.93
Step [20600/50000], Loss: 0.1150, Acc: 0.97
Step [20700/50000], Loss: 0.2134, Acc: 0.93
Step [20800/50000], Loss: 0.2322, Acc: 0.92
Step [20900/50000], Loss: 0.1703, Acc: 0.96
Step [21000/50000], Loss: 0.2530, Acc: 0.90
Step [21100/50000], Loss: 0.1375, Acc: 0.96
Step [21200/50000], Loss: 0.2048, Acc: 0.95
Step [21300/50000], Loss: 0.1746, Acc: 0.96
Step [21400/50000], Loss: 0.2453, Acc: 0.95
Step [21500/50000], Loss: 0.1539, Acc: 0.98
Step [21600/50000], Loss: 0.2388, Acc: 0.91
Step [21700/50000], Loss: 0.2739, Acc: 0.91
Step [21800/50000], Loss: 0.3101, Acc: 0.91
Step [21900/50000], Loss: 0.2273, Acc: 0.91
Step [22000/50000], Loss: 0.2275, Acc: 0.93
Step [22100/50000], Loss: 0.2803, Acc: 0.90
Step [22200/50000], Loss: 0.3033, Acc: 0.90
Step [22300/50000], Loss: 0.2143, Acc: 0.96
Step [22400/50000], Loss: 0.1619

Step [26800/50000], Loss: 0.1420, Acc: 0.97
Step [26900/50000], Loss: 0.2654, Acc: 0.91
Step [27000/50000], Loss: 0.1850, Acc: 0.96
Step [27100/50000], Loss: 0.2077, Acc: 0.94
Step [27200/50000], Loss: 0.2682, Acc: 0.91
Step [27300/50000], Loss: 0.1161, Acc: 0.97
Step [27400/50000], Loss: 0.2050, Acc: 0.91
Step [27500/50000], Loss: 0.1329, Acc: 0.98
Step [27600/50000], Loss: 0.2937, Acc: 0.93
Step [27700/50000], Loss: 0.1610, Acc: 0.94
Step [27800/50000], Loss: 0.0931, Acc: 0.97
Step [27900/50000], Loss: 0.2165, Acc: 0.94
Step [28000/50000], Loss: 0.1277, Acc: 0.97
Step [28100/50000], Loss: 0.1055, Acc: 0.97
Step [28200/50000], Loss: 0.2992, Acc: 0.90
Step [28300/50000], Loss: 0.1816, Acc: 0.97
Step [28400/50000], Loss: 0.2448, Acc: 0.92
Step [28500/50000], Loss: 0.2253, Acc: 0.93
Step [28600/50000], Loss: 0.2175, Acc: 0.94
Step [28700/50000], Loss: 0.1308, Acc: 0.97
Step [28800/50000], Loss: 0.2453, Acc: 0.93
Step [28900/50000], Loss: 0.2032, Acc: 0.92
Step [29000/50000], Loss: 0.1536

Step [33400/50000], Loss: 0.0992, Acc: 0.98
Step [33500/50000], Loss: 0.3224, Acc: 0.92
Step [33600/50000], Loss: 0.1580, Acc: 0.96
Step [33700/50000], Loss: 0.1622, Acc: 0.94
Step [33800/50000], Loss: 0.1429, Acc: 0.98
Step [33900/50000], Loss: 0.1329, Acc: 0.96
Step [34000/50000], Loss: 0.1529, Acc: 0.97
Step [34100/50000], Loss: 0.1233, Acc: 0.96
Step [34200/50000], Loss: 0.2140, Acc: 0.94
Step [34300/50000], Loss: 0.0941, Acc: 0.99
Step [34400/50000], Loss: 0.1997, Acc: 0.95
Step [34500/50000], Loss: 0.1154, Acc: 0.98
Step [34600/50000], Loss: 0.1001, Acc: 0.97
Step [34700/50000], Loss: 0.1284, Acc: 0.94
Step [34800/50000], Loss: 0.1793, Acc: 0.93
Step [34900/50000], Loss: 0.2286, Acc: 0.94
Step [35000/50000], Loss: 0.1942, Acc: 0.94
Step [35100/50000], Loss: 0.1157, Acc: 0.98
Step [35200/50000], Loss: 0.1693, Acc: 0.95
Step [35300/50000], Loss: 0.1591, Acc: 0.95
Step [35400/50000], Loss: 0.0901, Acc: 0.98
Step [35500/50000], Loss: 0.1880, Acc: 0.96
Step [35600/50000], Loss: 0.2035

Step [40000/50000], Loss: 0.2912, Acc: 0.90
Step [40100/50000], Loss: 0.1567, Acc: 0.96
Step [40200/50000], Loss: 0.1580, Acc: 0.99
Step [40300/50000], Loss: 0.1925, Acc: 0.94
Step [40400/50000], Loss: 0.1399, Acc: 0.97
Step [40500/50000], Loss: 0.1548, Acc: 0.93
Step [40600/50000], Loss: 0.1789, Acc: 0.94
Step [40700/50000], Loss: 0.1674, Acc: 0.96
Step [40800/50000], Loss: 0.1749, Acc: 0.96
Step [40900/50000], Loss: 0.1456, Acc: 0.98
Step [41000/50000], Loss: 0.1054, Acc: 0.98
Step [41100/50000], Loss: 0.1460, Acc: 0.96
Step [41200/50000], Loss: 0.0542, Acc: 0.99
Step [41300/50000], Loss: 0.0508, Acc: 1.00
Step [41400/50000], Loss: 0.0664, Acc: 0.98
Step [41500/50000], Loss: 0.1781, Acc: 0.96
Step [41600/50000], Loss: 0.1315, Acc: 0.94
Step [41700/50000], Loss: 0.1522, Acc: 0.95
Step [41800/50000], Loss: 0.1334, Acc: 0.97
Step [41900/50000], Loss: 0.0678, Acc: 0.98
Step [42000/50000], Loss: 0.1663, Acc: 0.95
Step [42100/50000], Loss: 0.2034, Acc: 0.93
Step [42200/50000], Loss: 0.0643